In [2]:
import os
import platform
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
from src.functions import get_matrices_from_dfs, normalize_proportion_ratios
from joblib import Parallel, delayed
from tqdm import tqdm
import statsmodels.formula.api as smf

In [10]:
if platform.system() == "Darwin":
    source_path = '/Volumes/alexandr/smenon/2022-07-13_Glioblastoma/processed_files'
    target_path = '/Volumes/mklein/FDA_project/data/Lx_Glioblastoma'
else:
    source_path = '/g/alexandr/smenon/2022-07-13_Glioblastoma/processed_files'
    target_path = '/home/mklein/FDA_project/data/Lx_Glioblastoma'


samples = os.listdir(source_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/alexandr/smenon/2022-07-13_Glioblastoma/processed_files'

In [3]:
# this is how martijn performed the calculations
from src.functions import CELL_PRE


def cell_normalization_Rappez_adata(sampling_prop_matrix, sampling_spec_matrix, adata, raw_adata, sampling_prop_threshold = 0.3, sampling_spec_threshold = 0):
    
    # filter out pixels with little overlap with any cell (thus sum of all overlaps)
    pixel_sampling_prop_keep = sampling_prop_matrix.sum(axis = 0) > sampling_prop_threshold
    # filter out pixels with low contributions to a cell
    pixel_sampling_spec_keep = sampling_spec_matrix > sampling_spec_threshold

    sampling_prop_matrix_filtered = sampling_prop_matrix.sum(axis = 0) * pixel_sampling_prop_keep
    sampling_spec_matrix_filtered = sampling_spec_matrix * pixel_sampling_spec_keep

    sum_prop_matrix = sampling_prop_matrix_filtered.replace(to_replace=0, value=pd.NA)

    # create dataframe for results
    norm_ion_intensities = ad.AnnData(obs=pd.DataFrame({'cell_id': sampling_prop_matrix.index}), var=adata.var)
    norm_spots = adata.to_df().multiply(1/sum_prop_matrix, axis=0).replace(np.nan, 0)
    
    cor_df = sampling_spec_matrix_filtered.replace(np.nan, 0).dot(norm_spots)

    norm_ion_intensities.X = cor_df.multiply(1/sampling_spec_matrix_filtered.sum(axis=1), axis=0)
    norm_ion_intensities.obs.index = norm_ion_intensities.obs.cell_id.map(lambda x: x.replace(CELL_PRE, ""))

    norm_ion_intensities = norm_ion_intensities[raw_adata.obs_names]
    norm_ion_intensities.obs = raw_adata.obs
    
    return norm_ion_intensities

In [4]:
from src.functions import PIXEL_PRE, correct_intensities_quantile_regression_parallel, get_molecule_normalization_factors
import statistics as st


def correct_sample(sample):
    
    sample_path = os.path.join(source_path, sample, "analysis")

    if not os.path.exists(os.path.join(target_path, sample)):
        os.makedirs(os.path.join(target_path, sample))


    files = {
        'config': '../config.json',
        'sm_matrix': 'ablation_mark_analysis/spatiomolecular_adata.h5ad',
        'overlap_regions': 'overlap_analysis1/overlap.regions.csv',
        'mark_regions': 'overlap_analysis1/ablation_mark.regions.csv',
        'cell_regions': 'overlap_analysis1/cell.regions.csv',
        'cell_sm_matrix': 'single_cell_analysis/spatiomolecular_adata.h5ad',
    }

    project_files = {k: os.path.join(sample_path, v) for k, v in files.items()}

    cell_regions = pd.read_csv(project_files['cell_regions'])
    mark_regions = pd.read_csv(project_files['mark_regions'])
    overlap_regions = pd.read_csv(project_files['overlap_regions'])

    overlap_matrix, sampling_spec_matrix = get_matrices_from_dfs(mark_area = mark_regions, cell_area = cell_regions, marks_cell_overlap = overlap_regions)

    sm_matrix = sc.read(os.path.join(sample_path, files['sm_matrix']))
    sm_matrix.obs_names = PIXEL_PRE + sm_matrix.obs_names
    cell_sm_matrix = sc.read(os.path.join(sample_path, files['cell_sm_matrix']))

    total_pixel_overlap, full_pixel_intensities_median = get_molecule_normalization_factors(sm_matrix.to_df(), overlap_matrix, method= st.median)

    corrected_intensities = correct_intensities_quantile_regression_parallel(sm_matrix.to_df(), total_pixel_overlap, full_pixel_intensities_median, reference_ions=sm_matrix.var_names, n_jobs=10)

    corr_sm_matrix = sm_matrix.copy()
    corr_sm_matrix.X = corrected_intensities

    corr_cell_sm_matrix = cell_normalization_Rappez_adata(sampling_prop_matrix=overlap_matrix, sampling_spec_matrix=sampling_spec_matrix, adata=corr_sm_matrix, raw_adata=cell_sm_matrix)

    corr_cell_sm_matrix.write(os.path.join(target_path, sample, 'cells_spatiomolecular_adata_corrected.h5ad'))
    cell_sm_matrix.write(os.path.join(target_path, sample, 'cells_spatiomolecular_adata.h5ad'))
    
    return True


Parallel(n_jobs=8)(delayed(correct_sample)(sample) for sample in tqdm(samples))








 23%|██▎       | 8/35 [00:00<00:00, 64.23it/s]/home/mklein/.conda/envs/ion_suppression/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
  0%|          | 0/2100 [00:00<?, ?it/s]/home/mklein/.conda/envs/ion_suppression/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
 46%|████▌     | 16/35 [00:10<00:13,  1.36it/s]s]

KeyboardInterrupt: 

In [ ]:
# pd.concat([corr_sm_matrix.to_df().replace(0, np.nan)['C10H20NO8P+K'], sm_matrix.to_df().replace(0, np.nan)['C10H20NO8P+K'], total_pixel_overlap.replace(0, np.nan)], axis=1).dropna()

In [ ]:
import os


samples = [s for s in os.listdir(target_path) if s != ".DS_Store"]
# samples = [samples[0], samples[20]]

adata_dict = {}
adata_cor_dict = {}

condition_name = 'dataset_3'

for sample in samples:
    sample_path = os.path.join(target_path, sample)
    files = {
        'cell_sm_matrix': 'cells_spatiomolecular_adata.h5ad',
        'corr_cell_sm_matrix': 'cells_spatiomolecular_adata_corrected.h5ad',
    }

    project_files = {k: os.path.join(sample_path, v) for k, v in files.items()}

    adata = sc.read(project_files['cell_sm_matrix'])
    adata_cor = sc.read(project_files['corr_cell_sm_matrix'])

    adata_dict[sample] = adata
    adata_cor_dict[sample] = adata_cor



In [ ]:
def split_dataset_info(adata):
    split = adata.obs['dataset'].str.split("_", expand=True)
    adata.obs[['dataset_' + str(col) for col in split.columns]] = split

In [ ]:
def concat_batches(adata_dict):
    adata = ad.concat(adata_dict, label='well', index_unique="_", merge="same")
    sc.tl.pca(adata)
    sc.external.pp.bbknn(adata, batch_key='well')
    split_dataset_info(adata)
    return adata

adata = concat_batches(adata_dict)
adata_cor = concat_batches(adata_cor_dict)


In [ ]:
adata.obs[condition_name].value_counts()

In [ ]:
adata.write(os.path.join(target_path, "batch_sm_matrix.h5ad"))
adata_cor.write(os.path.join(target_path, "corrected_batch_sm_matrix.h5ad"))

 46%|████▌     | 16/35 [00:29<00:13,  1.36it/s]